In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import scipy
from scipy.optimize import curve_fit
from scipy.stats import poisson
from inspect import signature


from pandas_datareader import data as pdr
import datetime as dt
import yfinance as yfin

from mpmath import mp
mp.dps = 50

# Multiple Parameters Calibration

When dealing with multiple parameters of non-smooth functional data, it's well-known that multiple local minima may exist. To address this issue, two main solutions are commonly employed, to the best of my knowledge:

> Bayesian Methods: Constructing a Markov Chain Monte Carlo (MCMC) chain is a feasible approach, especially when a dynamics model for the underlying asset is available. This dynamics, expressed as a distribution of log-returns, serves as the Likelihood function. You can specify a prior based on your intuition or a frequentist estimate of the mean and variance, though the latter approach implies the exploitation of data twice.

Here's a simplified example of MCMC to calibrate a Geometric Brownian Motion (GBM) model (toy problem):
$$X_{t}=\bigg(r-\frac{\sigma^{2}}{2}\bigg)t + \sigma W_{t}\to X_{t}\sim N\bigg(\bigg(r-\frac{\sigma^{2}}{2}\bigg)t,\sigma^{2}t\bigg).$$

$$\text{Prior } \sigma\sim \pi(\sigma),\hspace{1cm} \text{Likelihood } X_{t}|\sigma\sim N\bigg(\bigg(r-\frac{\sigma^{2}}{2}\bigg)t,\sigma^{2}t\bigg)=:L(X_{t}|\sigma).$$

$$\text{By Bayes, }\pi(\sigma|X_{t})\propto L(X_{t}|\sigma)\times \pi(\sigma)=:Kernel(\sigma).$$

- Initialize $\sigma^{(0)}$.
- For i in range(N):

    - Propose a candidate $\sigma^{(i),c}=\sigma^{(i-1)}+\text{ perturbation }$.
    
    - Draw from a uniform distribution $U(0,1)$ the sample u.
    
    - If $$\frac{Kernel(\sigma^{(i),c})}{Kernel(\sigma^{(i-1)})}>u\hspace{1cm} \sigma^{(i)}=\sigma^{(i),c},$$
    otherwise $$\sigma^{(i)}=\sigma^{(i-1)}.$$

- The chain $\{\sigma^{(burn-in)},...,\sigma^{(N)}\}$ represents $N-(burn-in-ratio)$ samples from the posterior of $\sigma$ for N large enough. Adopt the mean of those samples as an estimation of the parameter.

> Gradient-based Optimization: Algorithms like ADAM and similar methods are frequently employed. However, it's essential to deeply investigate your cost function and ensure adaptivity of the learning rate to prevent being trapped in local minima.

$\textbf{HOWEVER,}$ here I propose a convergent but inefficient alternative: a brute-force raw algorithm to calibrate parameters.

In [3]:
# optimization setup
iterations = 10**4 #higher is better
step = 0.02 #lower is better with higher number of iterations

In [4]:
def optimizer(f, T, data):
    
    sig = signature(f)
    n_params = len(sig.parameters) -  1
    
    init = np.zeros(n_params)
    chain = [init]
    errors =  10**6

    choices = np.random.choice([1, 0], size=(iterations, n_params))
    
    for i in range(iterations):
        temp = np.abs(chain[-1] + choices[i] *  step)
        error = np.average((f(T, *temp) - data)**2)
        
        if error < errors:
            chain.append(temp)
            errors = error
            
    return chain[-1]

In [5]:
class stock:
    def __init__(self, T, data, N, rf,S0):
        self.T = T
        interp = scipy.interpolate.CubicSpline(self.T[0::10],data[0::10])
        self.data = interp(self.T)
        self.N = N
        self.rf = rf
        self.S0 = S0
        self.M = T.shape[0]
        
    def underlying_simulation(self, T, params, model):
        X = np.zeros((self.N, self.M))
        dt = self.T[1]-self.T[0]

        if model == 'gbm':
            flag =  1

            sigma = np.array(params)
            corr = +0.5*(sigma**2)

        elif model == 'merton':
            flag =  2
            sigma, lambda_, muJ, sigmaJ = params
            corr = +0.5*(sigma**2) + lambda_ * (np.exp(0.5*sigmaJ**2 +  muJ) -  1)

        elif model == 'kou':
            flag =  3
            sigma, lambda_, p, lamp, lamm = params
            corr = +0.5*(sigma**2) +  lambda_ * (p / (lamp -1) - (1 - p) / (lamm +  1))

        else:
            raise ValueError('Error in model specification. Use gbm, merton or kou.')
        
        drift = rf-corr

        Z = np.random.randn(self.N, self.M)

        if flag ==  1:
            for i in range(self.M-1):
                X[:, i+1] = X[:, i] + np.ones(self.N)*drift * dt + np.sqrt(dt) * sigma * Z[:, i]

        elif flag ==  2:
            Ndt = poisson.rvs(lambda_ * dt, size=(self.M,self.N))
            for i in range(self.M-1):
                X[:, i+1] = X[:, i] + np.ones(self.N)*drift * dt + np.sqrt(dt) * sigma * Z[:, i]
                for j in range(self.N):
                    if Ndt[i,j] >  0:
                        Y = sum(muJ + sigmaJ * np.random.normal(0, 1, Ndt[i,j]))
                        X[j, i+1] = X[j, i+1] + Y

        elif flag ==  3:
            Ndt = poisson.rvs(lambda_ * dt, size=(self.M,self.N))
            for i in range(self.M-1):
                X[:, i+1] = X[:, i] + np.ones(self.N)*drift * dt + np.sqrt(dt) * sigma * Z[:, i]
                for j in range(self.N):
                    for jj in range(Ndt[i,j]):
                        if np.random.uniform(0,1,1) < p:  # positive
                            Y = np.random.exponential(lamp)
                        else:
                            Y = -np.random.exponential(lamm)
                        X[j, i+1] = X[j, i+1] + Y

        S = self.S0 * np.exp(np.average(X,axis=0))
        return S
    
    
    def calibrate(self, model):
        if model == 'gbm':
            def wrapper(T, sigma):
                return self.underlying_simulation(T, [sigma], model)
        if model == 'merton':
            def wrapper(T, sigma, lambda_, muJ, sigmaJ):
                par = [sigma, lambda_, muJ, sigmaJ]
                return self.underlying_simulation(T, par, model)
        if model == 'kou':
            def wrapper(T, sigma, lambda_, p, lamp, lamm):
                par = [sigma, lambda_, p, lamp, lamm]
                return self.underlying_simulation(T, par, model)

        popt = optimizer(wrapper, self.T, self.data)

        return popt

# Calibration

In [6]:
# model specifications
model = 'kou'
rf = 0.05 # to be observed

# data
yfin.pdr_override()
aapl = np.array(pdr.get_data_yahoo('AAPL', start='2023-08-21', end='2024-02-21')['Close'])
T = np.linspace(0,0.5,aapl.shape[0])

# calibration
asset = stock(T, aapl, 1000, rf, aapl[0])
par = asset.calibrate(model)

print(par)

[*********************100%***********************]  1 of 1 completed
[0.28 0.26 0.1  0.24 0.22]
